In [1]:
cd /content/drive/My Drive/FireFriday

/content/drive/My Drive/FireFriday


In [2]:
ls

FireKeras.ipynb    radam_submission.csv   test_df.csv.zip
FirePytorch.ipynb  sample_submission.csv  train_df.csv
kaggle.json        test_df.csv            train_df.csv.zip


## 전처리...

In [ ]:
import pandas as pd
sample_submission = pd.read_csv("sample_submission.csv")
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")

In [ ]:
X = train.iloc[:,1:].values / 255
y = train.iloc[:,0].values
X_test = test.iloc[:,1:].values / 255

## Dataset이랑 Loader 정의하기

In [ ]:
from torch.utils.data import Dataset, DataLoader

class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

class TestDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        return X

In [ ]:
traindataloader = DataLoader(TrainDataset(X, y), batch_size=128, shuffle=True, num_workers=4)
testdataloader = DataLoader(TestDataset(X_test, y=None), batch_size=4, shuffle=False, num_workers=4)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 10)
        self.elu = nn.ELU()
        self.dropout1 = torch.nn.Dropout(p=0.2)
        self.dropout2 = torch.nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(F.relu(x))
        x = self.dropout1(x)
        x = self.fc3(self.elu(x))
        x = self.dropout2(x)
        x = self.fc4(x)
        return x

net = Net()

In [32]:
net

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
)

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=2e-3, weight_decay=1e-3)

In [37]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [38]:
net.to(device)

Net(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=10, bias=True)
  (elu): ELU(alpha=1.0)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.2, inplace=False)
)

In [46]:
num_epochs = 20
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(traindataloader):
        inputs, labels = data
        inputs = torch.tensor(inputs, device=device).float()
        labels = torch.tensor(labels, device=device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # print(type(inputs), type(labels), type(outputs))
        # forward + backward + optimize
        outputs = net(inputs)
        # print(type(inputs), type(labels), type(outputs))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    print("Epoch : {} loss: {}".format(epoch, running_loss))
print('Finished Training')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


Epoch : 0 loss: 1.891492336988449
Epoch : 1 loss: 2.031160682439804
Epoch : 2 loss: 3.1714176516979933
Epoch : 3 loss: 0.7157164290547371
Epoch : 4 loss: 0.5604358650743961
Epoch : 5 loss: 0.7958797551691532
Epoch : 6 loss: 1.4181657303124666
Epoch : 7 loss: 1.6583141535520554
Epoch : 8 loss: 0.44541674479842186
Epoch : 9 loss: 0.16661260649561882
Epoch : 10 loss: 0.25935184210538864
Epoch : 11 loss: 0.18625089153647423
Epoch : 12 loss: 3.391363959759474
Epoch : 13 loss: 1.2553639523684978
Epoch : 14 loss: 1.5438310224562883
Epoch : 15 loss: 0.630724124610424
Epoch : 16 loss: 0.9427573196589947
Epoch : 17 loss: 0.7635397091507912
Epoch : 18 loss: 0.17023862153291702
Epoch : 19 loss: 0.01919320970773697
Finished Training


In [56]:
correct = 0
total = 0
net.eval()
preds = []
with torch.no_grad():
    for data in testdataloader:
        inputs = data
        inputs = torch.tensor(inputs, device=device).float()
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
          preds.append(pred.cpu().numpy())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [57]:
preds[:10]

[array(8),
 array(0),
 array(5),
 array(3),
 array(8),
 array(1),
 array(9),
 array(6),
 array(6),
 array(0)]

In [60]:
import numpy as np
preds = np.array(preds)
print(preds.shape)

(18000,)


In [62]:
sample_submission["Category"] = pd.Series(preds)
sample_submission.head()

,Id,Category
0,57808,8
1,4960,0
2,35755,5
3,15543,3
4,48968,8


In [ ]:
sample_submission.to_csv("torch_submission.csv", index=False)